# 使用ICEL构建简单的LLM应用

本教程的目标：
1. 使用语言模型
2. 使用 PromptTemplates 和 OutputParser
3. 使用 LangChain 表达式语言 （LCEL） 将组件链接在一起
4. 使用 LangSmith 调试和跟踪应用程序
5. 使用 LangServe 部署应用程序

### 安装langchain

In [ ]:
!pip install langchain

### LangSmith
检查整个项目中的内部情况

设置langsmith的环境变量

In [ ]:
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="..."
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")


# import getpass
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()  # 输入API_KEY

## 使用语言模型

In [7]:
# 提供多种库的支持，参考：https://python.langchain.com/v0.2/docs/tutorials/llm_chain/
!pip install -qU langchain-openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.9 requires langchain-core<0.2,>=0.1.26, but you have langchain-core 0.2.10 which is incompatible.
langchain-community 0.0.24 requires langchain-core<0.2,>=0.1.26, but you have langchain-core 0.2.10 which is incompatible.


In [8]:
import getpass
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# os.environ["OPENAI_API_KEY"] = getpass.getpass()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("OPENAI_API_BASE")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

直接使用模型

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-55addf47-44d1-41b6-a23a-d51015fd4988-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

## 输出解析器（OutputParser）

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [11]:
# 方式一：传统方式
result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

In [12]:
# 方式二：使用chain
chain = model | parser
chain.invoke(messages)

'Ciao!'

## 提示模版（Prompt Templates）

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
system_template = "Translate the following into {language}:"

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [ ]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

In [ ]:
# 直接访问消息
result.to_messages()

## 使用LCEL将组件链接在一起

In [ ]:
chain = prompt_template | model | parser

In [ ]:
chain.invoke({"language": "italian", "text": "hi"})

## LangServe服务

In [13]:
!pip install "langserve[all]"

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 1.6 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: sse-starlette
    Found existing installation: sse-starlette 2.0.0
    Uninstalling sse-starlette-2.0.0:
      Successfully uninstalled sse-starlette-2.0.0


详见LangServe文件夹

## 客户端使用

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})